<h2 style="text-align: center;">Job Application Status Classifier</h2>


In [2]:
import pandas as pd
import string
import re
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
pd.set_option('display.max_colwidth', None)
import spacy
spacy_model = spacy.load("en_core_web_md")

2024-11-15 00:32:59.109721: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 00:32:59.119254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731652379.130070  206537 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731652379.133273  206537 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 00:32:59.144930: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Read dataset

In [3]:
# Read the dataset
SRC = '../data/job_application_emails-v2.csv'
df = pd.read_csv(SRC)
df.head()

,ID,Text,Label1,Label2,From
0,1931c0fd0aba6dd7,"Hi Samiul,\n\nThank you for your interest in our Summer 2025 Internship program\n\nUnfortunately, we have decided to move forward with other candidates.\n\nTo be clear, this has no bearing on future applications here and we invite you to apply again in the future.\n\nThank you again for your interest in PDT Partners.\n\nBest,\n\nPDT Talent & Culture Team",Rejection,Generic,no-reply@pdtpartners.com
1,1931726956eba16f,"Hi Samiul,\n\nThank you for your interest in a role at Twilio! We just received your application.\n\nWe know that job hunting can be stressful. Our intention is to be transparent about our hiring process to help you succeed. On that note, let's talk a bit about our hiring process.\n\nReviewing Your Application\n\nRather than relying on bots and algorithms, Twilio has human recruiters reviewing every application, so it can take a little longer than automated screening. Also, there are no loose ends with us: we'll let you know where you stand either way.\n\nOur Interview Process\n\nIf we like what we see, a recruiting team member will reach out to discuss the opportunity. The next steps could include a few rounds of interviews taking from 2-4 weeks, sometimes longer. While this email doesn't guarantee an interview, we encourage you to check out our Interview Prep Guide ( https://drive.google.com/file/d/15uGbg6AfEbBPt5keqi0NW2ezBPWjaBAD/view ).\n\nEven if you don't move forward with us, it's got some great tips for active job seekers. And to get a feel for the Twilio experience, check out our culture page ( https://www.twilio.com/company/culture ) and follow our LinkedIn ( https://www.linkedin.com/company/twilio-inc-/ ).\n\nThanks again for your interest in building the future of communications!\n\nThank you,\n\n-Twilio Recruiting Team\n\n#WeBuildAtTwilio\n\n( https://www.instagram.com/webuildattwilio/ )\n\n( https://twitter.com/webuildattwilio?lang=en )\n\n( https://www.linkedin.com/company/twilio-inc-/ )",Application Received,NaN,no-reply@twilio.com
2,1931540a2ce12bd4,"\n\n\n\nHi Samiul, Thank you for your interest in the Software Engineering LMTS\n role at Salesforce.Your time and effort are greatly appreciated. Unfortunately, we are no longer hiring\n for this position. Thank you for considering Salesforce in your job search. We will\n keep your resume on file and consider you for other positions that match your skills.With gratitude,The Salesforce Recruiting TeamP.S. This is an auto-generated message. Please do not reply to this email.Subject: Samiul Mushfik ‎- JR269090 Software Engineering LMTS (CA7813597)Click here to view the notification details.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThis email was intended for mushf001@d.umn.edu\n\n\n\n\nsalesforce.com inc. · Salesforce Tower, 415 Mission Street, 3rd Floor, San Francisco, CA 94105\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n",Rejection,No longer accepting,"""workday-support@salesforce.com"" <salesforce@myworkday.com>"
3,1930d4fa93d81e44,"4dE8b1Jx5k2yU7sM3n9TqC6LzV4wP0rA8g5QX\n\nHi Samiul,\n\nThank you so much for your interest in internship opportunities with Snowflake and completing the HackerRank coding challenge. After reviewing your score and background/skillset for a potential fit, we have unfortunately decided not to proceed with your candidacy for the Software Engineer Intern (Infrastructure Automation) - Spring 2025 opening this time around.\n\nBut nothing is forever! We strongly encourage you to reapply for upcoming seasons that are a fit with your background and schedule. We look forward to connecting again.\n\nSpeaking of connecting, we’d like to encourage you to follow Snowflake on https://www.linkedin.com/company/snowflake-computing/ and https://instagram.com/_snowflake_inc?igshid=YmMyMTA2M2Y= They’re both great places to keep up with all things Snowflake.\n\nThanks again for your interest in Snowflake and we hope to see you again in the future!\n\nRegards,\nThe University Recrui

### Pre-processing

In [4]:
def convert_to_lowercase(text: str) -> str:
    return text.lower()

def remove_line_breaks(text: str) -> str:
    return re.sub(r"\s+", " ", text.strip())

def remove_punctuation(text: str) -> str:
    return text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))

def remove_nonalphanumeric_characters(text: str) -> str:
    return re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

def extract_main_body(text: str) -> str:
    start_phrases = ["Samiul", "Samiul Mushfik"]
    end_phrases = [
        "Thank you,\n", "Best,", r"(?i)(?:Yours\s)?sincerely,", "Best Regards,", 
        "With gratitude,", r"Thanks[,!]", "Regards,", "Cheers,", "\\*\\*", 
        r"Thanks again (and)?", "connect soon", "Respectfully,", 
        "The Redfin Recruiting Team", "All the best,", "In the meantime,", 
        "consideration", "Best wishes,", r"Thanks again for your interest?,", 
        "Kind regards,", "Good luck!", "Take care,", "- Shipt", 
        "Thank you once again", "Wish you all the best", "We wish you success", 
        "Sent from", "Speaking of connecting"
    ]

    pattern = (
        r"(?:" + "|".join(re.escape(start.lower()) + r"[,\s]?" for start in start_phrases) +
        r")(.+?)(?:" + "|".join(re.escape(end.lower()) for end in end_phrases) + r"|$)"
    )

    matches = re.findall(pattern, text, re.IGNORECASE)
    return matches[0] if matches else text

def prepend(text: str) -> str:
    start_phrases = ["Samiul", "Samiul Mushfik"]
    pattern = r"(?:{})".format("|".join(re.escape(phrase) for phrase in start_phrases))
    return "Samiul, " + text if not re.search(pattern, text, re.IGNORECASE) else text

    
def remove_links(text: str) -> str:
    pattern = r'\(?https?://[^\s)]+\)?'
    return re.sub(pattern, '', text)

def remove_stopwords(text: str) -> str:
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)


tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")
ner = pipeline("ner", model=model, tokenizer=tokenizer)

def remove_company_names(text: str) -> str:
    results = ner(text)
    words = []
    for i in range(len(results)):
        if results[i]['entity'] == 'B-ORG':
            words.append(results[i]['word'])
            j = i + 1
            while j < len(results) and results[j]['entity'] == 'I-ORG':
                words.append(results[j]['word'])
                j += 1
            break
        
    if words:
        company = ".?".join(words).replace('##', '')
        return re.sub(company, '', text, flags=re.IGNORECASE)
    return text

def remove_datetime(text: str) -> str:
    doc = spacy_model(text)
    date_time_entities = [ent.text for ent in doc.ents if ent.label_ in ["DATE", "TIME"]]
    for entity in date_time_entities:
        text = re.sub(entity, '', text, flags=re.IGNORECASE)
    return text


def tokenize_and_lemmatize(text: str) -> str:
    doc = spacy_model(text)
    return " ".join([token.lemma_ for token in doc])
    

def remove_short_words(text: str, min_length: int = 2) -> str:
    return " ".join(word for word in text.split() if len(word) >= min_length)
    

/home/csgrads/mushf001/anaconda3/envs/application-status-classifier/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exact

In [5]:
processed_df = df.copy()

processed_df['Text'] = processed_df['Text'].apply(remove_line_breaks) \
    .apply(prepend) \
        .apply(extract_main_body) \
            .apply(remove_company_names) \
                .apply(convert_to_lowercase) \
                    .apply(remove_links) \
                        .apply(remove_nonalphanumeric_characters) \
                            .apply(remove_punctuation) \
                                .apply(remove_stopwords) \
                                    .apply(remove_datetime) \
                                        .apply(tokenize_and_lemmatize) \
                                            .apply(remove_short_words)
                                        
                                            
            
processed_df.head()

,ID,Text,Label1,Label2,From
0,1931c0fd0aba6dd7,thank interest internship program unfortunately decide move forward candidate clear bear future application invite apply future thank interest,Rejection,Generic,no-reply@pdtpartners.com
1,1931726956eba16f,thank interest role receive application know job hunt stressful intention transparent hiring process help succeed note let talk bit hire process review application rather rely bot algorithm human recruiter review every application take little long automate screening also loose end we let know stand either way interview process like see recruiting team member reach discuss opportunity next step could include round interview take sometimes long email guarantee interview encourage check interview prep guide even move forward we get great tip active job seeker get feel experience check culture page follow linkedin thank interest build future communication thank recruit team webuilda,Application Received,NaN,no-reply@twilio.com
2,1931540a2ce12bd4,thank interest software engineering lmts role time effort greatly appreciate unfortunately long hire position thank consider job search keep resume file consider position match skill,Rejection,No longer accepting,"""workday-support@salesforce.com"" <salesforce@myworkday.com>"
3,1930d4fa93d81e44,thank much interest internship opportunity complete hackerrank code challenge review score background skillset potential fit unfortunately decide proceed candidacy software engineer intern infrastructure automation open time around nothing forever strongly encourage reapply upcoming season fit background schedule look forward connect,Rejection,Feedback,Snowflake Hiring Team <no-reply@ashbyhq.com>
4,1930cae04d845673,thank apply software engineer backend team position receive application review soon possible background look like good match reach coordinate next step,Application Received,NaN,no-reply@doordash.com


In [13]:

# processed_df.to_csv('../data/processed_df.csv')

In [8]:
# processed_df[processed_df['Text'] == '']
# processed_df['From'].to_clipboard
# processed_df['From'].to_csv('../data/from.csv')

In [9]:
# from IPython.display import display
# from ipywidgets import Output

# output = Output(layout={'border': '1px solid black', 'height': '300px', 'overflow_y': 'scroll'})

# with output:
#     # Your long output code here
#     processed_df.head()
    
# display(output)
# processed_df.head()


In [10]:
# processed_df.to_csv("../data/processed_df.csv", index=False)
# text = df.iloc[67]['Text']
# text = remove_line_breaks(text)
# text = prepend(text)
# text = extract_main_body(text)
# text = remove_company_names(text)
# text = convert_to_lowercase(text)
# text = prepend(text)
# text = extract_main_body(text)
# text = remove_company_names(text)
# text = convert_to_lowercase(text)
# text = remove_line_breaks(text)
# text = remove_links(text)
# text = remove_nonalphanumeric_characters(text)
# text = remove_punctuation(text)
# text = remove_stopwords(text)
# text 

In [11]:
# from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForTokenClassification

# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")

# ner = pipeline("ner", model=model, tokenizer=tokenizer)
# text = "Thank you for your interest in the Software Engineering LMTS role at Salesforce.Your time and effort are greatly appreciated. Unfortunately, we are no longer hiring for this position. Thank you for considering Salesforce in your job search. We will keep your resume on file and consider you for other positions that match your skills."

# results = ner(text)
# # companies = [result['word'] for result in results if result['entity'] in ['B-ORG', 'I-ORG']]
# # for result in results: 
# #     if result['entity'] in ['B-ORG', 'I-ORG']:
# #         print(result['word'], result['entity'])
# # print("Found companies:", companies)

# words = []
# for i in range(len(results)):
#     if results[i]['entity'] == 'B-ORG':
#         words.append(results[i]['word'])
#         j = i + 1
#         while j < len(results) and results[j]['entity'] == 'I-ORG':
#             words.append(results[j]['word'])
#             j += 1
#         break
    
#     print(results[i]['word'], results[i]['entity'])
    
# # company = ".?".join(words).replace('##', '')
# # print(company)
# if words:
#     company = ".?".join(words).replace('##', '')
#     print(company)
#     print(re.sub(company, '', text, flags=re.IGNORECASE))
# # print(re.match(r"Door.?D.?ash", 'Doordash\'s', re.IGNORECASE))
# # escaped_company = re.escape(company)
# # print(re.sub(company, '', text, flags=re.IGNORECASE))

# # Process results to join subword tokens
# # companies = []
# # current_company = ""
# # for result in results:
# #     if result['entity'].startswith("B-ORG"):  # Beginning of an organization
# #         if current_company:
# #             companies.append(current_company)  # Append the last found organization
# #         current_company = result['word']
# #     elif result['entity'].startswith("I-ORG") and current_company:
# #         # Concatenate if it's part of the same organization
# #         if result['word'].startswith("##"):
# #             current_company += result['word'][2:]  # Remove the "##" for subword
# #         else:
# #             current_company += " " + result['word']
# #     else:
# #         if current_company:
# #             companies.append(current_company)
# #             current_company = ""
# # if current_company:
# #     companies.append(current_company)  # Append any remaining organization

# # print("Found companies:", companies)